In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import numpy as np
from itertools import combinations, groupby
from collections import Counter
from pathlib import Path
from apriori_algo import AprioriAlgo
apriori = AprioriAlgo()

In [2]:
orders = pd.read_csv('../input/all_orders.csv', index_col = 0)
cluster_df = pd.read_csv('../input/cluster_df')
products = pd.read_csv('../input/products.csv')
aisles = pd.read_csv('../input/aisles.csv')
departments = pd.read_csv('../input/departments.csv')

c:\users\akshaysharma\hvacproakshay\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
orders = orders.sort_values(['order_id','product_id'])
orders

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,aisle,department,user_id,cluster
30264547,1,10246,3,0,83,4,fresh vegetables,produce,112108,1
30264545,1,11109,2,1,108,16,other creams cheeses,dairy eggs,112108,1
30264548,1,13176,6,0,24,4,fresh fruits,produce,112108,1
30264544,1,22035,8,1,21,16,packaged cheese,dairy eggs,112108,1
30264550,1,43633,5,1,95,15,canned meat seafood,canned goods,112108,1
...,...,...,...,...,...,...,...,...,...,...
23108244,3421083,21162,3,0,92,18,baby food formula,babies,25247,1
23108238,3421083,24852,9,1,24,4,fresh fruits,produce,25247,1
23108247,3421083,35211,5,0,92,18,baby food formula,babies,25247,1
23108242,3421083,39678,6,1,74,17,dish detergents,household,25247,1


In [4]:
# separate order data by cluster and transform into expected format of group association function

cluster_orders = {}
for x in range(0,6):
    i = orders[orders['cluster'] == x]
    i = i.set_index('order_id')['product_id'].rename('item_id')
    cluster_orders[x] = i

In [11]:
# run each cluster series through association rule function and obtain association rules at item level
REBUILD_TEST_DATA = False
test_data_path = "../input/associationRule/group0_aisle_rules.csv"
if REBUILD_TEST_DATA or not Path(test_data_path).exists():
    group_association_rules_dic = {}
    for x in range(0,6):
        group_association_rules_dic[x] = apriori.association_rules(cluster_orders[x],.0001)

True